In [ ]:
import networkx as nx
from z3 import Solver, Bool, Sum, If
import matplotlib.pyplot as plt

def criaGrafo():
    graph = nx.DiGraph()
    edges = [(0,1), (0,3), (1,3), (2,1), (2,5), (3,5), (4,3)]
    graph.add_edges_from(edges)
    return graph

def solve(graph):
    solver = Solver()
    N = len(graph.nodes())
    
    # Criação de variáveis para as arestas
    E = {}
    for a, b in graph.edges():
        E[(a, b)] = Bool(f"E[{a},{b}]")
    
    # Restrições de conectividade
    for o in range(N):
        for d in range(N):
            if o != d:
                paths = list(nx.all_simple_paths(graph, source=o, target=d))
                if paths:
                    # Pelo menos um caminho deve estar disponível entre 'o' e 'd'
                    solver.add(Sum([If(all(E[(p[i], p[i+1])] for i in range(len(p) - 1)), 1, 0) for p in paths]) >= 1)
    
    # Maximizar o número de arestas removidas mantendo a conectividade
    num_edges = len(graph.edges())
    solver.add(Sum([If(E[(a, b)], 1, 0) for a, b in graph.edges()]) <= num_edges - 1)
    
    # Resolver o problema
    if solver.check() == z3.sat:
        model = solver.model()
        print("> Solution is optimal.")
        
        # Criar o grafo de saída
        resultado_grafo = nx.DiGraph()
        for (a, b) in graph.edges():
            if model[E[(a, b)]]:
                resultado_grafo.add_edge(a, b)
        
        return resultado_grafo
    else:
        print("Not feasible.")
        return None

# Criar o grafo com as arestas fornecidas
graph = criaGrafo()
resultado_grafo = solve(graph)

# Desenhar o grafo resultante
if resultado_grafo:
    nx.draw(resultado_grafo, with_labels=True, node_color='lightblue', arrows=True)
    plt.show()
